In [2]:
from dotenv import load_dotenv
import os 

load_dotenv(r"../.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_KEY[:5]

'sk-pr'

In [3]:
import datasets
from langchain.docstore.document import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]

c:\Projects\RAG_PoC\agents_course_hf\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%pip install --upgrade --quiet rank_bm25

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain_community.retrievers import BM25Retriever
from langchain.tools import Tool 

bm25_retriever = BM25Retriever.from_documents(docs)

def extract_text(query: str) -> str: 
    """Retrieves detailed information about gala guests based on their name or relations."""
    results = bm25_retriever.invoke(query)
    if results:
        return "\n\n".join([doc.page_content for doc in results[:3]])
    else:
        return "No matching guest information found." 
    
guest_info_tool = Tool(
    name="guest_info_retriever", 
    func=extract_text,
    description="Retrieves detailed information about gala guests based on their name or relation."
)

In [6]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages 
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage 
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph 
from langgraph.prebuilt import tools_condition 
from langchain_openai import ChatOpenAI 


llm = ChatOpenAI(temperature=0)

chat = llm 
tools = [guest_info_tool]
chat_with_tools = chat.bind_tools(tools)

# Gererate the AgentState and Agent graph

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

# The graph
builder = StateGraph(AgentState)

# Define nodes: thses do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition,)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

messages = [HumanMessage(content="Tell me about our guest named 'Lady Ada Lovelace'.")]
response = alfred.invoke({"messages": messages})

print("Alfred's Response:")
print(response['messages'][-1].content)

Alfred's Response:
Lady Ada Lovelace is your best friend. She is an esteemed mathematician and renowned for her pioneering work in mathematics and computing. She is often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. You can reach her at ada.lovelace@example.com.


In [11]:
%pip install -qU duckduckgo-search langchain-community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# Web search
from langchain_community.tools import DuckDuckGoSearchRun 

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke("Who's the current President of France?")
print(results)

Emmanuel Macron (born December 21, 1977, Amiens, France) is a French banker and politician who was elected president of France in 2017. Macron was the first person in the history of the Fifth Republic to win the presidency without the backing of either the Socialists or the Gaullists, and he was France's youngest head of state since Napoleon I.He was reelected in 2022, becoming the first ... Emmanuel Macron is the current President of France. Emmanuel Macron was born on December 21, 1977, in Amiens, France. He is a former investment banker who founded the centrist En Marche! Movement in 2016. Macron won the presidency in 2017. He made history by being the first to win without backing from major parties. This is a list of current heads of state and heads of government.In some cases, mainly in presidential systems, one leader is head of state and head of government.In other cases, mainly in semi-presidential and parliamentary systems, the head of state and the head of government are diff

In [15]:
# Creating a Custom tool for weather information
# For the example, we will use a dummy API

from langchain.tools import Tool 
import random 

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""

    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Sunny", "temp_c": 25},
        {"condition": "Cloudy", "temp_c": 20},
        {"condition": "Snowy", "temp_c": -5},
    ]

    data = random.choice(weather_conditions) 
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# Initialize the tool

weather_info_tool = Tool(
    name="weather_info_tool",
    func=get_weather_info, 
    description="Fetches dummy weather information for a given location."
)

In [17]:
# Hub stats tool for infulential AI Builders

from langchain.tools import Tool 
from huggingface_hub import list_models 

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models: 
            model = models[0]
            return f"Most downloaded model by {author} is {model.id} with {model.downloads} downloads." 
        else: 
            return f"No models found for author {author}." 
    except Exception as e: 
        return f"Error fetching models for {author}: {str(e)}" 
    
# Initialize the tool
hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches the most downloaded model from a specific author on the Hugging Face Hub." 
)

print(hub_stats_tool("facebook"))

C:\Users\axel.grille\AppData\Local\Temp\ipykernel_26740\2671613754.py:26: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(hub_stats_tool("facebook"))


Most downloaded model by facebook is facebook/esmfold_v1 with 23480812 downloads.
